In [16]:
### moviepy を用いてトリミング（エラーのため断念）####
# from moviepy.editor import *

# file_path = "movie/usImage1.mp4"
# X_start = 100
# Y_start = 100
# X_length = 500
# Y_length = 500
# save_path = "movie/usImage1_trim.mp4"
# video = (VideoFileClip(file_path).crop(x1=X_start, y1=Y_start, x2=X_length, y2=Y_length))
# video.write_videofile(save_path, fps=29)

In [17]:
#動画を１フレームずつ読み込み切り取り処理、その後再度動画化

import cv2

file_path = "movie/usImage1.mp4"
video = cv2.VideoCapture(file_path)

#properties
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
size = (width, height)
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(video.get(cv2.CAP_PROP_FPS))

print(height, width, size, frame_count, frame_rate)

fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
save = cv2.VideoWriter('movie/usImage_trim.mp4', fmt, frame_rate, size)

for i in range(frame_count):
  ret, frame = video.read()
  if ret==True:
    ### 処理 ###
    frame = frame[30:420,155:530]
    ############
    frame = cv2.resize(frame,(640,480))
    save.write(frame)
    cv2.imshow("img", frame)
    if cv2.waitKey(30)==27:
      break

save.release()
# video.release()
cv2.destroyAllWindows()

480 640 (640, 480) 360 12


In [19]:
COUNT = 100
criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 20, 0.03)

lk_params = dict(winSize=(10,10), maxLevel=4, criteria=criteria)
cap = cv2.VideoCapture('movie/usImage_trim.mp4')
#properties
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
size = (width, height)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
print(height, width, size, frame_count, frame_rate)

fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
save = cv2.VideoWriter('movie/usImage_flow2.mp4', fmt, frame_rate, size)

ret, frame = cap.read()
# print(ret, type(frame))
frame_pre = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
while True:
  ret, frame = cap.read()
  if ret == False:
    break
  frame_now = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  feature_pre = cv2.goodFeaturesToTrack(frame_pre, COUNT, 0.001, 5)
  if feature_pre is None:
    continue
  feature_now, status, err = cv2.calcOpticalFlowPyrLK(frame_pre, frame_now, feature_pre, None, **lk_params)
  for i in range(len(feature_now)):
    pre_x = int(feature_pre[i][0][0])
    pre_y = int(feature_pre[i][0][1])
    now_x = int(feature_now[i][0][0])
    now_y = int(feature_now[i][0][1])
    cv2.line(frame, (pre_x, pre_y), (now_x, now_y), (255,0,0), 3)
  cv2.imshow("img", frame)
  save.write(frame)
  frame_pre = frame_now.copy()
  key = cv2.waitKey(30)
  if key == 27:
    break

save.release()
cv2.destroyAllWindows()

480 640 (640, 480) 360 12
